Make that we are running the correct version of TensorFlow first

In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [2]:
import sys

assert sys.version_info >= (3, 6) # Python ≥3.6 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

# Train and use lenet5 for character predication

In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')

from pathlib import Path

import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow import keras

In [4]:
# Import traps: http://python-notes.curiousefficiency.org/en/latest/python_concepts/import_traps.html
from recognizer.datasets import EmnistDataset
from recognizer.networks import lenet5
from recognizer.networks import simple

Hyperparameters

In [5]:
batch_size = 256
epochs = 16

Setup dataset

In [6]:
emnist = EmnistDataset()

train_dataset = emnist.train_dataset.shuffle(1024).repeat().batch(batch_size)
test_dataset = emnist.test_dataset.batch(batch_size)

Download path: /Users/mostafagazar/Work/notebooks/line-reader/data/cache/datasets/matlab.zip
Processing data...
Balancing train dataset...
Target max number of images per class: 11256
Dataset ready, with 336163 training entries and 116323 test entries


In [7]:
train_dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None, 1)), types: (tf.float32, tf.uint8)>

In [8]:
(x_train, y_train), = train_dataset.take(1)
input_shape = x_train[0].shape
print(f"x shape: {x_train.shape}, model input shape: {input_shape}")

x shape: (256, 28, 28, 1), model input shape: (28, 28, 1)


## Quickly fit one batch and that everything is working as expected check 

In [9]:
# (x_test, y_test), = test_dataset.take(1)

# # model = lenet5(input_shape=input_shape, number_of_classes=emnist.number_of_classes)
# model = simple(input_shape=input_shape, number_of_classes=emnist.number_of_classes)

# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# model.fit(x=x_train, y=y_train, epochs=5)
# model.evaluate(x_test, y_test)

# Model training

[Get started with TensorFlow 2.0 for experts](https://www.tensorflow.org/alpha/tutorials/quickstart/advanced)

In [10]:
model_checkpoints_path = Path("../recognizer/ckpts/character_model")
model_checkpoints_path.mkdir(parents=True, exist_ok=True)
model_save_path = Path("../recognizer/weights/character_model.h5")

In [11]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

# initial_learning_rate = 0.01
# learning_rate_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=100000,
#     decay_rate=0.96,
#     staircase=True)

# optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate_schedule)

In [12]:
# initial_learning_rate = 0.01
# learning_rate_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=100000,
#     decay_rate=0.96,
#     staircase=True)

# learning_rate_schedule.ExponentialDecay(0.05, 10, 0.96)

# optimizer = keras.optimizers.Adam(learning_rate=learning_rate_schedule)

# for step in range(100, 100000):
#     print(float(optimizer._decayed_lr(step)))

In [13]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [14]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [15]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [16]:
# model = lenet5(input_shape=input_shape, number_of_classes=emnist.number_of_classes)
model = simple(input_shape=input_shape, number_of_classes=emnist.number_of_classes)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 62)                7998      
Total params: 108,478
Trainable params: 108,478
Non-trainable params: 0
_________________________________________________________________


In [17]:
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, model_checkpoints_path, max_to_keep=3)
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print(f"Restored from {manager.latest_checkpoint}")
else:
    print("Initializing from scratch.")

Initializing from scratch.


In [19]:
for epoch in range(1, epochs + 1):
    for images, labels in train_dataset:
        train_step(images, labels)
    
    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)
    
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
        save_path = manager.save()
        print(f"💾 Saved checkpoint for step {int(ckpt.step)}: {save_path}")
        
    print(f"Epoch {epoch}, "\
#           f"Current learning rate: {optimizer._lr}, "\
          f"Loss: {train_loss.result()}, Accuracy: {train_accuracy.result()*100}, "\
          f"Test Loss: {test_loss.result()}, Test Accuracy: {test_accuracy.result()*100}")

About to train


KeyboardInterrupt: 

# Save the model

In [ ]:
# model.save(model_save_path)

# Predict

In [ ]:
model.trainable = False